In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from typing import List

'''
None || 2000
'''
nb = None
nb_trains = nb
nb_valids = nb
nb_tests = None

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/Projects/JPX_Tokyo_Stock/working

/content/drive/MyDrive/Projects/JPX_Tokyo_Stock/working


In [ ]:
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 7.5 MB/s 
     |████████████████████████████████| 209 kB 86.1 MB/s 
     |████████████████████████████████| 81 kB 11.4 MB/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 
     |████████████████████████████████| 112 kB 64.1 MB/s 
     |████████████████████████████████| 49 kB 7.2 MB/s 
     |████████████████████████████████| 146 kB 65.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=f5434d638cde266a4f4940cd27f79902591d4ffb407543a2f3ae6eaf09b8a3c5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
from data_utils import *

In [ ]:
train_df = pd.read_csv('../input/1_after_datapreprocessing/train_full.csv', parse_dates=['Date'], nrows = nb_trains)
valid_df = pd.read_csv('../input/1_after_datapreprocessing/valid_full.csv', parse_dates=['Date'], nrows = nb_valids)
test_df = pd.read_csv('../input/1_after_datapreprocessing/test_full.csv', parse_dates=['Date'], nrows = nb_tests)
y_test_df = pd.read_csv('../input/1_after_datapreprocessing/y_test_full.csv', parse_dates=['Date'], nrows = nb_tests)

### check info

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4617191 entries, 0 to 4617190
Data columns (total 13 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Date              datetime64[ns]
 1   SecuritiesCode    int64         
 2   Volume            int64         
 3   AdjustmentFactor  float64       
 4   ExpectedDividend  float64       
 5   SupervisionFlag   bool          
 6   Target            float64       
 7   CloseT2           float64       
 8   Open              float64       
 9   High              float64       
 10  Low               float64       
 11  Close             float64       
 12  CloseT1           float64       
dtypes: bool(1), datetime64[ns](1), float64(9), int64(2)
memory usage: 427.1 MB


In [ ]:
train_df.head(20)

,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Open,High,Low,Close,CloseT1
0,2017-01-04,1301,31400,1.0,-1.0,False,0.000730,2740.0,2734.0,2755.0,2730.0,2742.0,2738.0
1,2017-01-04,1332,2798500,1.0,-1.0,False,0.012324,575.0,568.0,576.0,563.0,571.0,568.0
2,2017-01-04,1333,270800,1.0,-1.0,False,0.006154,3270.0,3150.0,3210.0,3140.0,3210.0,3250.0
3,2017-01-04,1376,11300,1.0,-1.0,False,0.011053,1555.0,1510.0,1550.0,1510.0,1550.0,1538.0
4,2017-01-04,1377,150800,1.0,-1.0,False,0.003026,3315.0,3270.0,3350.0,3270.0,3330.0,3305.0
5,2017-01-04,1379,77300,1.0,-1.0,False,0.005169,2139.0,2105.0,2147.0,2101.0,2143.0,2128.0
6,2017-01-04,1381,1300,1.0,-1.0,False,-0.009326,1912.0,1950.0,1960.0,1949.0,1960.0,1930.0
7,2017-01-04,1407,147000,1.0,-1.0,False,-0.003437,882.0,857.0,877.0,851.0,866.0,885.0
8,2017-01-04,1414,119600,1.0,-1.0,False,0.000000,5030.0,4940.0,5060.0,4935.0,5050.0,5030.0
9,2017-01-04,1417,347500,1.0,-1.0,False,-0.007463,1064.0,1051.0,1063.0,1048.0,1053.0,1072.0


In [ ]:
valid_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233748 entries, 0 to 233747
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              233748 non-null  datetime64[ns]
 1   SecuritiesCode    233748 non-null  int64         
 2   Volume            233748 non-null  int64         
 3   AdjustmentFactor  233748 non-null  float64       
 4   ExpectedDividend  233748 non-null  float64       
 5   SupervisionFlag   233748 non-null  bool          
 6   Target            233748 non-null  float64       
 7   CloseT2           233748 non-null  float64       
 8   Open              233748 non-null  float64       
 9   High              233748 non-null  float64       
 10  Low               233748 non-null  float64       
 11  Close             233748 non-null  float64       
 12  CloseT1           233748 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(9), int64(2)
memory 

In [ ]:
valid_df.head(20)

,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Open,High,Low,Close,CloseT1
0,2021-12-06,1305,104900,1.0,-1.0,False,0.005013,2105.0,2061.5,2065.0,2041.5,2048.0,2094.5
1,2021-12-06,1306,1427480,1.0,-1.0,False,0.005793,2083.5,2037.0,2041.0,2018.5,2025.0,2071.5
2,2021-12-06,1308,110000,1.0,-1.0,False,0.005367,2060.5,2016.0,2019.5,1997.5,2004.5,2049.5
3,2021-12-06,1309,323,1.0,-1.0,False,0.005927,44130.0,43400.0,43710.0,43030.0,43190.0,43870.0
4,2021-12-06,1311,2030,1.0,-1.0,False,0.006300,974.4,952.1,953.4,943.5,947.5,968.3
5,2021-12-06,1312,27,1.0,-1.0,False,0.015174,20740.0,20315.0,20315.0,20260.0,20260.0,20430.0
6,2021-12-06,1313,60,1.0,-1.0,False,0.014121,3950.0,4020.0,4020.0,3810.0,3810.0,3895.0
7,2021-12-06,1319,3000,1.0,-1.0,False,0.015284,372.0,364.0,364.0,364.0,364.0,366.4
8,2021-12-06,1320,28860,1.0,-1.0,False,0.011389,29750.0,28825.0,28840.0,28510.0,28750.0,29415.0
9,2021-12-06,1321,98437,1.0,-1.0,False,0.010348,29780.0,28890.0,28900.0,28560.0,28810.0,29475.0


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229952 entries, 0 to 229951
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              229952 non-null  datetime64[ns]
 1   SecuritiesCode    229952 non-null  int64         
 2   Volume            229952 non-null  int64         
 3   AdjustmentFactor  229952 non-null  float64       
 4   ExpectedDividend  229952 non-null  float64       
 5   SupervisionFlag   229952 non-null  bool          
 6   Open              229952 non-null  float64       
 7   High              229952 non-null  float64       
 8   Low               229952 non-null  float64       
 9   Close             229952 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(6), int64(2)
memory usage: 16.0 MB


In [ ]:
test_df.head(20)

,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Open,High,Low,Close
0,2021-12-06,1301,8900,1.0,-1.0,False,2982.0,2982.0,2965.0,2971.0
1,2021-12-06,1332,1360800,1.0,-1.0,False,592.0,599.0,588.0,589.0
2,2021-12-06,1333,125900,1.0,-1.0,False,2368.0,2388.0,2360.0,2377.0
3,2021-12-06,1375,81100,1.0,-1.0,False,1230.0,1239.0,1224.0,1224.0
4,2021-12-06,1376,6200,1.0,-1.0,False,1339.0,1372.0,1339.0,1351.0
5,2021-12-06,1377,77000,1.0,-1.0,False,3185.0,3215.0,3180.0,3200.0
6,2021-12-06,1379,48500,1.0,-1.0,False,1890.0,1898.0,1885.0,1889.0
7,2021-12-06,1381,1600,1.0,-1.0,False,3200.0,3200.0,3150.0,3150.0
8,2021-12-06,1407,246700,1.0,-1.0,False,6100.0,6290.0,6010.0,6220.0
9,2021-12-06,1413,9200,1.0,-1.0,False,2185.0,2211.0,2181.0,2190.0


In [ ]:
y_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229952 entries, 0 to 229951
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   SecuritiesCode  229952 non-null  int64         
 1   Date            229952 non-null  datetime64[ns]
 2   CloseT1         229952 non-null  float64       
 3   CloseT2         229952 non-null  float64       
 4   Target          229952 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 8.8 MB


In [ ]:
y_test_df.head(20)

,SecuritiesCode,Date,CloseT1,CloseT2,Target
0,1301,2021-12-06,3065.0,3055.0,-0.003263
1,1332,2021-12-06,556.0,551.0,-0.008993
2,1333,2021-12-06,2409.0,2385.0,-0.009963
3,1375,2021-12-06,1264.0,1245.0,-0.015032
4,1376,2021-12-06,1395.0,1399.0,0.002867
5,1377,2021-12-06,3285.0,3295.0,0.003044
6,1379,2021-12-06,1925.0,1906.0,-0.009870
7,1381,2021-12-06,3165.0,3165.0,0.000000
8,1407,2021-12-06,6340.0,6310.0,-0.004732
9,1413,2021-12-06,2250.0,2255.0,0.002222


### Calc Scores

In [ ]:
def calc_spread_return_per_day(df, portfolio_size: int = 200, toprank_weight_ratio: float = 2):
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): spread return
    """
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    buf = df.groupby('Date').apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

def add_rank(df: pd.DataFrame, y_pred):
    df["Pred"] = y_pred
    df["Rank"] = df.groupby("Date")["Pred"].rank(ascending=False, method="first") - 1 
    df = df.drop("Pred", axis= 1)
    return df

In [ ]:
def calc_score(df: pd.DataFrame, y_pred: pd.DataFrame, y_true: pd.DataFrame, render_info= True):
    feature_df = df.copy()
    feature_df = add_rank(feature_df, y_pred)
    feature_df['Target'] = y_true
    score, buf = calc_spread_return_sharpe(feature_df)
    if render_info:
        print(f'score -> {score}\nmean -> {buf.mean()}\nstd -> {buf.std()}')
    return score

In [ ]:
calc_score(test_df, y_test_df["Target"], y_test_df["Target"])

score -> 4.687380919991682
mean -> 15.446241603409499
std -> 3.2952819211963917


4.687380919991682

### Cross Validation

In [ ]:
featureCols = train_df.drop(["Target", "CloseT1", "CloseT2"], axis = 1).columns
featureCols

Index(['Date', 'SecuritiesCode', 'Volume', 'AdjustmentFactor',
       'ExpectedDividend', 'SupervisionFlag', 'Open', 'High', 'Low', 'Close'],
      dtype='object')

In [ ]:
X_train = train_df[featureCols]
X_train["Date"] = X_train["Date"].astype(str)
y_train = train_df["Target"]

X_valid = valid_df[featureCols]
X_valid["Date"] = X_valid["Date"].astype(str)
y_valid = valid_df["Target"]

X_test = test_df[featureCols]
X_test["Date"] = X_test["Date"].astype(str)
y_test = y_test_df["Target"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
calc_score(X_test, y_test, y_test)

score -> 4.687380919991682
mean -> 15.446241603409499
std -> 3.2952819211963917


4.687380919991682

#### Optuna tuning

In [ ]:
def objective(
    trial,
    random_state=22,
    n_jobs=-1,
):
    # XGBoost parameters
    params = {
        'tree_method':'gpu_hist',
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "objective": "reg:squarederror",
        "n_estimators": 1000,
        "max_depth": trial.suggest_int("max_depth", 4, 18),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.6, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": random_state,
        "n_jobs": n_jobs,
        "early_stopping_rounds": 50,
        "eval_metric": "rmse"
        
    }
    
    model = xgb.XGBRegressor(**params) 

    model.fit(
        X_train.drop("Date", axis = 1),
        y_train,
        eval_set=[(X_valid.drop("Date", axis = 1), y_valid)],
        verbose=0,
    )
    y_pred = pd.DataFrame(model.predict(X_valid.drop("Date", axis = 1)), columns =['Target'])
    return calc_score(X_valid, y_pred, y_valid, render_info= False)

In [ ]:
# break point
1/0

ZeroDivisionError: ignored

In [ ]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-06-15 14:12:12,290] A new study created in memory with name: no-name-cc37ad2d-0ac3-41ec-ad7a-1b2b938ad846
[I 2022-06-15 14:13:15,942] Trial 0 finished with value: 0.3787554351195579 and parameters: {'max_depth': 13, 'learning_rate': 0.009949828287618982, 'colsample_bytree': 0.4142624548860624, 'subsample': 0.7510682112526379, 'alpha': 0.9991101119871342, 'lambda': 3.074272408552209e-07, 'min_child_weight': 101.6765897770275}. Best is trial 0 with value: 0.3787554351195579.
[I 2022-06-15 14:13:40,989] Trial 1 finished with value: 0.3854950644551866 and parameters: {'max_depth': 6, 'learning_rate': 0.03944966287629853, 'colsample_bytree': 0.2804464183339555, 'subsample': 0.7485646953790775, 'alpha': 3.682057293336046, 'lambda': 1.6880175547694884e-08, 'min_child_weight': 28.795532941603714}. Best is trial 1 with value: 0.3854950644551866.
[I 2022-06-15 14:14:47,079] Trial 2 finished with value: 0.3845222891847453 and parameters: {'max_depth': 13, 'learning_rate': 0.01505882288257

Number of finished trials: 30
Best trial: {'max_depth': 11, 'learning_rate': 0.03386141413143176, 'colsample_bytree': 0.2094792165837191, 'subsample': 0.6246244338919174, 'alpha': 0.018059173074789, 'lambda': 2.362467695251705e-06, 'min_child_weight': 26.655961486533872}
CPU times: user 26min 19s, sys: 24.6 s, total: 26min 43s
Wall time: 26min 34s


In [ ]:
best_params = {
  'max_depth': 11, 
  'learning_rate': 0.03386141413143176, 
  'colsample_bytree': 0.2094792165837191, 
  'subsample': 0.6246244338919,
  'lambda': 2.362467695251705e-06, 
  'min_child_weight': 26.655961486533872,
  'early_stopping_rounds': 50,
  'eval_metric': 'rmse',
  'n_estimators': 1000,
  'n_jobs': -1,
  'objective': 'reg:squarederror',
  'seed': 22,
  'tree_method': 'gpu_hist',
  'verbosity': 0
}

best_params

{'colsample_bytree': 0.2094792165837191,
 'early_stopping_rounds': 50,
 'eval_metric': 'rmse',
 'lambda': 2.362467695251705e-06,
 'learning_rate': 0.03386141413143176,
 'max_depth': 11,
 'min_child_weight': 26.655961486533872,
 'n_estimators': 1000,
 'n_jobs': -1,
 'objective': 'reg:squarederror',
 'seed': 22,
 'subsample': 0.6246244338919,
 'tree_method': 'gpu_hist',
 'verbosity': 0}

In [ ]:
# best_params = study.best_trial.params
# best_params['tree_method'] = 'gpu_hist'
# best_params['verbosity'] = 0
# best_params['objective'] = "reg:squarederror"
# best_params['n_estimators'] = 1000
# best_params['seed'] = 22
# best_params['n_jobs'] = -1
# best_params["early_stopping_rounds"] = 50
# best_params["eval_metric"] = "rmse"

# best_params

In [ ]:
%%time
model = xgb.XGBRegressor(**best_params) 

model.fit(
    X_train.drop("Date", axis = 1),
    y_train,
    eval_set=[(X_valid.drop("Date", axis = 1), y_valid)],
    verbose=0,
)

CPU times: user 48.5 s, sys: 1.09 s, total: 49.6 s
Wall time: 49.7 s


In [ ]:
%%time
y_pred = pd.DataFrame(model.predict(X_train.drop("Date", axis = 1)), columns =['Target'])
print(calc_score(X_train, y_pred, y_train, render_info= False))

0.4495844651242038
CPU times: user 10.1 s, sys: 711 ms, total: 10.9 s
Wall time: 10.8 s


In [ ]:
np.sqrt(mean_squared_error(y_pred, y_train))

0.026191447718626668

In [ ]:
%%time
y_pred = pd.DataFrame(model.predict(X_valid.drop("Date", axis = 1)), columns =['Target'])
print(calc_score(X_valid, y_pred, y_valid, render_info= False))

0.4755083462004289
CPU times: user 620 ms, sys: 30.1 ms, total: 650 ms
Wall time: 645 ms


In [ ]:
np.sqrt(mean_squared_error(y_pred, y_valid))

0.028124388605469668

In [ ]:
%%time
y_pred = pd.DataFrame(model.predict(X_test.drop("Date", axis = 1)), columns =['Target'])
print(calc_score(X_test, y_pred, y_test, render_info= False))

0.05669352816907126
CPU times: user 626 ms, sys: 33 ms, total: 659 ms
Wall time: 653 ms


In [ ]:
np.sqrt(mean_squared_error(y_pred, y_test))

0.024271618342734867

#### test pred official_test

In [ ]:
official_test_df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv', nrows = None, parse_dates=['Date'])
mini_train_after_dataPreprocessing = pd.read_csv('../input/1_after_datapreprocessing/mini_train.csv', parse_dates=['Date'], nrows = None)

In [ ]:
official_test_df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False
2,20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False
3,20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False
4,20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
3995,20211207_9990,2021-12-07,9990,526.0,535.0,524.0,535.0,57800,1.0,NaN,False
3996,20211207_9991,2021-12-07,9991,795.0,806.0,792.0,805.0,48500,1.0,NaN,False
3997,20211207_9993,2021-12-07,9993,1640.0,1640.0,1620.0,1620.0,6600,1.0,NaN,False
3998,20211207_9994,2021-12-07,9994,2437.0,2440.0,2423.0,2440.0,5200,1.0,NaN,False


In [ ]:
y_test_official = DataPreprocessing_for_official_test(official_test_df, mini_train_after_dataPreprocessing)
y_test_official

/content/drive/MyDrive/Projects/JPX_Tokyo_Stock/working/data_utils.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_return['ExpectedDividend'] = df_return['ExpectedDividend'].fillna(-1)


,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Open,High,Low,Close
0,2021-12-06,1301,8900,1.0,-1.0,False,2982.0,2982.0,2965.0,2971.0
1,2021-12-06,1332,1360800,1.0,-1.0,False,592.0,599.0,588.0,589.0
2,2021-12-06,1333,125900,1.0,-1.0,False,2368.0,2388.0,2360.0,2377.0
3,2021-12-06,1375,81100,1.0,-1.0,False,1230.0,1239.0,1224.0,1224.0
4,2021-12-06,1376,6200,1.0,-1.0,False,1339.0,1372.0,1339.0,1351.0
...,...,...,...,...,...,...,...,...,...,...
3995,2021-12-07,9990,57800,1.0,-1.0,False,526.0,535.0,524.0,535.0
3996,2021-12-07,9991,48500,1.0,-1.0,False,795.0,806.0,792.0,805.0
3997,2021-12-07,9993,6600,1.0,-1.0,False,1640.0,1640.0,1620.0,1620.0
3998,2021-12-07,9994,5200,1.0,-1.0,False,2437.0,2440.0,2423.0,2440.0


### Submission

In [ ]:
# import jpx_tokyo_market_prediction
# env = jpx_tokyo_market_prediction.make_env()
# iter_test = env.iter_test()
# for (df_test, options, financials, trades, secondary_prices, df_pred) in iter_test:
#     print(df_test.info())
#     print(df_test.head(10))
#     print(secondary_prices.head(10))
    
    
#     env_x_test = EDA_df_for_official_Test(df_test, train_df)

#     env_y_pred =  pd.DataFrame(model.predict(env_x_test.drop("Date", axis = 1)), columns =['Target'])
    
#     df_pred['Target'] = env_y_pred
#     df_pred = add_rank(df_pred, env_y_pred)
    
#     submission = df_pred[["Date", "SecuritiesCode", "Rank"]]  
#     print(submission.columns)
#     env.predict(submission)